In [1]:
import pandas as pd
import numpy as np

In [99]:
adf = pd.read_csv("mobiles1.csv", usecols =['name', 'price', 'rating','memory']) # android raw
idf = pd.read_csv("apple_products.csv", usecols=['Product Name','Mrp','Star Rating','Ram']) # iphone raw

adf['Ram'] = adf['memory'].str[:1]
adf = adf.drop(columns=['memory'])

idf['Ram'] = adf['Ram'].str[:1]

#   1 USD = 75.31 INR
adf['Price_USD']=adf.price/75.31
adf['Rating per dollar'] = adf.rating / adf.Price_USD

idf['Price_USD'] = idf['Mrp']/75.31
idf['Rating per dollar'] = idf['Star Rating'] /idf['Price_USD']

adf['Ram per Dollar'] = adf['Ram'].astype(int) /adf['Price_USD']
idf['Ram per Dollar'] = idf['Ram'].astype(int) /idf['Price_USD']

idf=idf.rename(columns={"Product Name": "name", "Mrp": "price",'Star Rating':'rating'})

df=adf.append(idf).reset_index()

df['storage'] = df['name'].str[-7:-3].str.extract('(\d+)')
df.dropna(subset=['storage'],inplace=True)
df['storage'] = df['storage'].astype(int)

bins = [0, .02, .04, .06, .08,0.1,.12,.14,.16]
labels =[1,2,3,4,5,6,7,8]
df['Rating per dollar binned'] = pd.cut(df['Rating per dollar'], bins,labels=labels)

df


,index,name,price,rating,Ram,Price_USD,Rating per dollar,Ram per Dollar,storage,Rating per dollar binned
0,0,"Redmi 8 (Ruby Red, 64 GB)",9999,4.4,4,132.771212,0.033140,0.030127,64,2
1,1,"Realme 5i (Aqua Blue, 64 GB)",10999,4.5,4,146.049661,0.030811,0.027388,64,2
2,2,"Realme 5i (Aqua Blue, 128 GB)",11999,4.5,4,159.328110,0.028244,0.025105,128,2
3,3,"Realme 5i (Forest Green, 128 GB)",11999,4.5,4,159.328110,0.028244,0.025105,128,2
4,4,"Realme C2 (Diamond Blue, 32 GB)",7499,4.4,3,99.575090,0.044188,0.030128,32,3
...,...,...,...,...,...,...,...,...,...,...
1041,57,"APPLE iPhone SE (Black, 64 GB)",39900,4.5,3,529.810118,0.008494,0.005662,64,1
1042,58,"APPLE iPhone 11 (Purple, 64 GB)",54900,4.6,6,728.986854,0.006310,0.008231,64,1
1043,59,"APPLE iPhone 11 (White, 64 GB)",54900,4.6,6,728.986854,0.006310,0.008231,64,1
1044,60,"APPLE iPhone 11 (Black, 64 GB)",54900,4.6,3,728.986854,0.006310,0.004115,64,1


^^^ new combined dataframe with a storage and binned  Rating per dollar columns.
I made the bins using the std of Rating per dollar found using .describe() these bins will be used in the third aggregation.


In [100]:
def range_(arr):
    return arr.max() - arr.min()

print('Sum:')
display(df.groupby('Ram')[['Rating per dollar']].sum())

print('Count:')
display(df.groupby('Ram')[['name']].count())

print('Max:')
display(df.groupby('Ram')[['Price_USD']].max())

print('Mean:')
display(df.groupby('Ram')[['Price_USD']].mean())

print('Range:')
display(df.groupby('Ram')[['Price_USD']].agg(range_))

Sum:


,Rating per dollar
Ram,
1,5.820556
2,6.558684
3,7.321666
4,6.115663
5,0.907194
6,3.369229
8,1.538705


Count:


,name
Ram,
1,98
2,133
3,204
4,254
5,12
6,199
8,138


Max:


,Price_USD
Ram,
1,1394.223875
2,1554.906387
3,1990.439517
4,1990.439517
5,92.935865
6,1857.655026
8,1857.655026


Mean:


,Price_USD
Ram,
1,162.306141
2,114.723304
3,183.391850
4,280.093148
5,52.765237
6,331.730689
8,460.363637


Range:


,Price_USD
Ram,
1,1361.041030
2,1528.362767
3,1937.591289
4,1913.437790
5,61.080866
6,1725.029877
8,1658.611074


^^^ The highest Rating per dollar value is at 3 GB.
The most common ram configuration is 4GB.
The 5 GB category seems to be an outlier as it always has a different value.
    


In [101]:

print('Sum:')
display(df.groupby('storage')[['Rating per dollar']].sum())

print('Count:')
display(df.groupby('storage')[['name']].count())

print('Max:')
display(df.groupby('storage')[['Price_USD']].max())

print('Mean:')
display(df.groupby('storage')[['Price_USD']].mean())

print('Range:')
display(df.groupby('storage')[['Price_USD']].agg(range_))


Sum:


,Rating per dollar
storage,
4,0.596191
8,4.294987
16,6.834869
32,8.271019
64,6.797825
128,4.002617
256,0.556971
512,0.277218


Count:


,name
storage,
4,9
8,60
16,126
32,214
64,282
128,272
256,65
512,10


Max:


,Price_USD
storage,
4,119.572434
8,119.373257
16,557.973709
32,351.865622
64,1554.906387
128,1724.870535
256,1857.655026
512,1990.439517


Mean:


,Price_USD
storage,
4,64.686702
8,55.946311
16,85.119496
32,117.595993
64,238.893849
128,387.772452
256,718.646620
512,1051.380959


Range:


,Price_USD
storage,
4,87.717435
8,86.190413
16,520.130129
32,297.556765
64,1477.904661
128,1592.245386
256,1618.775727
512,1963.895897


^^^ 64GB is the most common storage configuration. Price_USD seems to have a postive correlation with the amount of storage based on the mean aggregation.


In [102]:
print('Sum:')
display(df.groupby('Rating per dollar binned')[['Ram per Dollar']].sum())

print('Count:')
display(df.groupby('Rating per dollar binned')[['name']].count())

print('Max:')
display(df.groupby('Rating per dollar binned')[['Price_USD']].max())

print('Mean:')
display(df.groupby('Rating per dollar binned')[['Price_USD']].mean())

print('Range:')
display(df.groupby('Rating per dollar binned')[['Price_USD']].agg(range_))

Sum:


,Ram per Dollar
Rating per dollar binned,
1,6.585805
2,9.337717
3,4.912512
4,1.856060
5,1.428278
6,0.180816
7,0.150695
8,0.000000


Count:


,name
Rating per dollar binned,
1,380
2,370
3,172
4,68
5,31
6,2
7,2
8,0


Max:


,Price_USD
Rating per dollar binned,
1,1990.439517
2,218.961625
3,107.422653
4,67.587306
5,53.100518
6,33.182844
7,26.543620
8,NaN


Mean:


,Price_USD
Rating per dollar binned,
1,488.264402
2,152.681170
3,87.227830
4,56.380295
5,43.595290
6,33.182844
7,26.543620
8,NaN


Range:


,Price_USD
Rating per dollar binned,
1,1804.554508
2,145.943434
3,46.740141
4,28.150312
5,21.245519
6,0.000000
7,0.000000
8,NaN


^^^ The 8th bin was included to show  where the bins ended.
Most of the devices are in the first bin this because most of the device have a low rating per dollar.
It may appear that bin 2 has the highest but that is because bin 2 has the second highest count

Based on the aggregations I have done , the 4GB ram 64GB storage seems to be like the most common model. Another thing I have noticed is the higher up you go in specifications the more variance is present.

while this aggregation shows data on both sets(Iphone and Android) combined, in assignment 3 using the star ratings plot,
I believe that apple products are more consistent with their ratings and tend to score high.
But on the other hand their configurations are limiting and Android product can have higher performance specs and tend to have higher composite ratings like Ram per dollar and ratings per dollar.